## Transform ResStock Run Results to 15 SF prototypes (Elevate Energy)

In [219]:
import pandas as pd
import numpy as np
import os
print(f'Notebook path: {os.getcwd()}')

notebook path: /Users/lliu2/Documents/GitHub/OpenStudio-BuildStock/files


### Download results from S3:
://S3/resbldg-datasets/chicagoeui

In [438]:
# local path to downloaded results
iteration = '_05' # <----- options: 'base', '_01'~'_06'
result_path = '/Users/lliu2/Documents/Chicago retrofits/ResStock results/'
iteration_path = f'cookcnty_2007{iteration}/baseline' 
baseline_result = os.path.join(result_path,iteration_path,'results_up00.parquet')
print(f'Result and file storage path: {iteration_path}')


Result and file storage path: cookcnty_2007_05/baseline


In [439]:
res = pd.read_parquet(baseline_result)
res

,building_id,job_id,started_at,completed_at,completed_status,apply_upgrade.applicable,apply_upgrade.upgrade_name,apply_upgrade.reference_scenario,build_existing_model.ahs_region,build_existing_model.applicable,...,qoi_report.average_minimum_daily_use_overlap_kw,qoi_report.average_of_top_ten_highest_peaks_timing_cooling_hour,qoi_report.average_of_top_ten_highest_peaks_timing_heating_hour,qoi_report.average_of_top_ten_highest_peaks_use_cooling_kw,qoi_report.average_of_top_ten_highest_peaks_use_heating_kw,qoi_report.peak_magnitude_timing_kw,qoi_report.peak_magnitude_use_kw,total_gas_heating,total_elec_heating,total_elec_cooling
0,1,1,2020-09-04 04:39:57,2020-09-04 04:41:14,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,2.003829,17.000000,16.818182,19.942221,12.040607,5082.0,23.857504,863.708669,0.000000,3694.245539
1,2,8,2020-09-04 07:49:35,2020-09-04 07:51:11,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,3.083140,18.363636,18.272727,11.794582,14.653871,1580.0,16.703020,1398.826479,33.353869,0.000000
2,3,24,2020-09-04 04:23:57,2020-09-04 04:25:40,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,1.515194,14.727273,18.363636,20.060215,9.937320,4526.0,22.787277,590.819064,575.466005,4508.437193
3,4,14,2020-09-04 08:44:00,2020-09-04 09:00:43,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,17.648995,14.454545,15.909091,569.520464,546.842815,4521.0,829.815874,9010.745970,3288.128828,48508.825140
4,5,15,2020-09-04 08:01:15,2020-09-04 08:02:38,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,0.778731,11.363636,16.000000,20.016977,20.067025,4113.0,23.728755,300.076336,293.062380,664.157733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,39996,24,2020-09-04 06:17:05,2020-09-04 06:18:32,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,1.132146,13.272727,18.454545,20.104937,5.302718,4544.0,21.862077,1310.242623,106.125872,4940.309830
39996,39997,8,2020-09-04 07:10:01,2020-09-04 07:11:27,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,1.089958,16.181818,15.727273,11.932807,7.648369,4528.0,13.713101,1096.388459,899.186385,1375.216035
39997,39998,27,2020-09-04 05:03:59,2020-09-04 05:58:59,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,60.140791,16.545455,18.636364,334.572608,215.826476,4528.0,387.028435,9763.648134,9528.753286,70731.506955
39998,39999,36,2020-09-04 06:39:59,2020-09-04 06:41:24,Success,NaN,NaN,,"CBSA Chicago-Naperville-Elgin, IL-IN-WI",True,...,1.319488,15.545455,17.545455,10.477427,7.033878,3779.0,12.645119,557.082192,499.883275,2163.632252


In [440]:
### HVAC cols changed after HVAC restructuring (implemented in Run5)
if iteration in ['base','_01','_02','_03','_04']:
    cooling_col = 'build_existing_model.hvac_system_cooling_type'
else:
    cooling_col = 'build_existing_model.hvac_cooling_type'
if iteration in ['base','_01','_02','_03','_04']:
    heating_col = 'build_existing_model.hvac_system_heating_natural_gas'
else:
    heating_col = 'build_existing_model.hvac_heating_type_and_fuel'
print('selected')


selected


### Check housing charateristics distributions

In [441]:
proto_chars = ['build_existing_model.geometry_stories',
              'build_existing_model.geometry_wall_type',
              'build_existing_model.vintage_acs',
              cooling_col,
              heating_col,
              'build_existing_model.geometry_floor_area'
             ]

Ns = len(res.query('completed_status=="Success"')); N = len(res)
print(f'>>> ResStock - {iteration_path} - RESULT SUMMARY:\n')
print(f'  * {Ns} / {N} samples ran successfully, {N-Ns} failed, efficacy: {Ns/N:.1%} \n')

print('>>> Housing characteristics splits:\n')

Res_char = []
for i, char in enumerate(proto_chars,1):
    Nchar = res.groupby(char)['building_id'].count()
    Nchar = Nchar/Ns
    Nchar['N_failed'] = len(res[res[char].isnull()])
    print(f'  * [{i}] {Nchar}\n')
    
    ## append for export
    Nchar = Nchar.rename('fraction').to_frame()
    Nchar['housing_char'] = Nchar.index.name
    Res_char.append(Nchar)
    
Res_char = pd.concat(Res_char, axis=0)
Res_char.index.name = 'sub_char'
Res_char = Res_char.reset_index()
Res_char = Res_char[['housing_char','sub_char','fraction']]

Res_char.to_csv(os.path.join(result_path, iteration_path, 'baseline_housing_char_breakdown.csv'), index=False)
print(f'>>> file saved to:\n  {os.path.join(result_path, iteration_path)}')

>>> ResStock - cookcnty_2007_05/baseline - RESULT SUMMARY:

  * 39917 / 40000 samples ran successfully, 83 failed, efficacy: 99.8% 

>>> Housing characteristics splits:

  * [1] build_existing_model.geometry_stories
1            0.498960
2            0.295087
3            0.205952
N_failed    83.000000
Name: building_id, dtype: float64

  * [2] build_existing_model.geometry_wall_type
Masonry      0.699226
WoodStud     0.300774
N_failed    83.000000
Name: building_id, dtype: float64

  * [3] build_existing_model.vintage_acs
1940-59      0.235138
1960-79      0.254879
1980-99      0.137761
2000-09      0.076108
2010s        0.011699
<1940        0.284415
N_failed    83.000000
Name: building_id, dtype: float64

  * [4] build_existing_model.hvac_cooling_type
Central AC     0.444447
Heat Pump      0.050430
None           0.207831
Room AC        0.297292
N_failed      83.000000
Name: building_id, dtype: float64

  * [5] build_existing_model.hvac_heating_type_and_fuel
Electricity ASHP        

### Get results by building prototypes from Elevate Energy
#### Note: cannot add prototype tags to ResStock results as prototypes overlap in chars

In [442]:
## Elevate Energy prototype chars
groups = pd.read_csv(os.path.join(result_path,'Groups.csv'))
print('loaded')


loaded


In [443]:
### median - original
groups['count'] = np.NAN
groups['median gas'] = np.NAN
groups['median elec'] = np.NAN

for i, row in groups.iterrows():
    res_group_i = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]

    groups.iloc[i, groups.columns.get_loc('count')] = \
        len(res_group_i[res_group_i['completed_status']=="Success"])
    
    groups.iloc[i, groups.columns.get_loc('median gas')] = \
        res_group_i['simulation_output_report.total_site_natural_gas_therm'].median()

    groups.iloc[i, groups.columns.get_loc('median elec')] = \
        res_group_i['simulation_output_report.total_site_electricity_kwh'].median()


In [444]:
groups['Gas Diff-med(%)'] = ((groups['Non-normalized gas usage'] - groups['median gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-med(%)'] = ((groups['Non-normalized elec. usage'] - groups['median elec']) / groups['Non-normalized elec. usage']) * 100
print('computed')


computed


In [445]:
## adding mean values

groups['mean gas'] = np.NAN
groups['mean elec'] = np.NAN
for i, row in groups.iterrows():
    
    res_group_i = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    
    mean_gas = res_group_i['simulation_output_report.total_site_natural_gas_therm'].mean()
    groups.iloc[i, groups.columns.get_loc('mean gas')] = mean_gas

    mean_elec = res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    groups.iloc[i, groups.columns.get_loc('mean elec')] = mean_elec

groups['Gas Diff-mean(%)'] = ((groups['Non-normalized gas usage'] - groups['mean gas']) / groups['Non-normalized gas usage']) * 100
groups['Elec Diff-mean(%)'] = ((groups['Non-normalized elec. usage'] - groups['mean elec']) / groups['Non-normalized elec. usage']) * 100

groups[['HousingGroupNo','HousingGroupName','median gas','mean gas','median elec','mean elec']]


,HousingGroupNo,HousingGroupName,median gas,mean gas,median elec,mean elec
0,1,Masonry Post-1978 less than 2 stories,701.823110,760.723057,8710.662051,9743.608038
1,2,Masonry All Years Split Level,1082.151900,1179.969170,9109.631282,10426.850641
2,3,Masonry Post-1978 2 or more stories,1102.059252,1214.647922,11136.748672,13208.710534
3,4,Masonry 1942-1978 Less than 2 Stories,950.899719,1028.854140,8438.543606,9531.278067
4,5,Masonry 1942-1978 2 or more stories,1459.271962,1606.330236,10335.399225,11611.197149
5,6,Masonry Pre-1942 less than 2 stories,902.389870,951.463950,7973.776697,8876.944588
6,7,Masonry Pre-1942 2 or more stories,1388.303517,1462.572065,10182.996506,11829.399011
7,8,Frame Post-1978 Less than 2 stories,723.058443,805.087201,8700.472351,9895.694721
8,9,Frame Post-1978 Split Level,906.787695,1042.542349,9786.061754,11696.917235
9,10,Frame Post-1978 2 or more stories,1121.269569,1272.419140,11175.157751,13440.653925


In [446]:
## diagnostics (1)
# aggregate heating and cooling end uses in ResStock results
res['total_gas_heating'] = res['simulation_output_report.natural_gas_central_system_heating_therm'] +\
    res['simulation_output_report.natural_gas_heating_therm']

res['total_elec_heating'] = res['simulation_output_report.electricity_central_system_heating_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_heating_kwh'] +\
    res['simulation_output_report.electricity_fans_heating_kwh'] +\
    res['simulation_output_report.electricity_heating_kwh'] +\
    res['simulation_output_report.electricity_pumps_heating_kwh']

res['total_elec_cooling'] = res['simulation_output_report.electricity_central_system_cooling_kwh'] +\
    res['simulation_output_report.electricity_central_system_pumps_cooling_kwh'] +\
    res['simulation_output_report.electricity_cooling_kwh'] +\
    res['simulation_output_report.electricity_fans_cooling_kwh'] +\
    res['simulation_output_report.electricity_pumps_cooling_kwh']

print('computed')

computed


In [447]:
## diagnostics (2)
# get heating and cooling end uses to "groups"
groups['frac_gas_heating'] = np.NAN
groups['frac_elec_heating'] = np.NAN
groups['frac_elec_cooling'] = np.NAN
groups['size_heating_kbut_h'] = np.NAN
groups['size_cooling_kbut_h'] = np.NAN

for i, row in groups.iterrows():
    
    res_group_i = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    
    groups.iloc[i, groups.columns.get_loc('frac_gas_heating')] = \
        res_group_i['total_gas_heating'].mean()/res_group_i['simulation_output_report.total_site_natural_gas_therm'].mean()

    groups.iloc[i, groups.columns.get_loc('frac_elec_heating')] = \
        res_group_i['total_elec_heating'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    groups.iloc[i, groups.columns.get_loc('frac_elec_cooling')] = \
        res_group_i['total_elec_cooling'].mean()/res_group_i['simulation_output_report.total_site_electricity_kwh'].mean()
    
    groups.iloc[i, groups.columns.get_loc('size_heating_kbut_h')] = \
        res_group_i['simulation_output_report.size_heating_system_kbtu_h'].mean()
    
    groups.iloc[i, groups.columns.get_loc('size_cooling_kbut_h')] = \
        res_group_i['simulation_output_report.size_cooling_system_kbtu_h'].mean()

print('computed')

computed


In [448]:
## diagnostics (3)
# get heating & cooling and floor area char breakdowns

House = []
House_chars = [cooling_col, heating_col,
              'build_existing_model.geometry_floor_area',
              'build_existing_model.insulation_unfinished_attic']

for i, row in groups.iterrows():
    
    res_group_i = res[res['build_existing_model.geometry_stories'].isin(row['Stories'].split(',')) & \
        res['build_existing_model.geometry_wall_type'].isin(row['WallType'].split(',')) & \
        res['build_existing_model.vintage_acs'].isin(row['Vintage'].split(',')) & \
        res['build_existing_model.geometry_building_type_recs'].isin(['Single-Family Detached'])]
    
    Ns = len(res_group_i.query('completed_status=="Success"'))
    Hi = []
    for char in House_chars:
        Nchar = res_group_i.groupby(char)['building_id'].count()/Ns # fractions
        Nchar = Nchar.rename('fraction').to_frame()
        Nchar['housing_char'] = Nchar.index.name
        Hi.append(Nchar)
    
    Hi = pd.concat(Hi, axis=0)
    Hi.index.name = 'sub_char'
    Hi = Hi.reset_index()
    Hi['HousingGroupNo'] = row['HousingGroupNo']
    
    House.append(Hi)
    
House = pd.concat(House, axis=0)
House = pd.pivot_table(House, values='fraction', index=['HousingGroupNo'],
                    columns=['housing_char','sub_char'], aggfunc=np.sum)

print('computed')

computed


In [449]:
## Combine "groups" + "House" into "Out" for export
out = groups.drop(['Stories','WallType','Vintage'], axis=1)
out = pd.concat([out.set_index('HousingGroupNo')], axis=1, keys=['none']).join(House)
out

none  \
                                     HousingGroupName   
HousingGroupNo                                          
1               Masonry Post-1978 less than 2 stories   
2                       Masonry All Years Split Level   
3                 Masonry Post-1978 2 or more stories   
4               Masonry 1942-1978 Less than 2 Stories   
5                 Masonry 1942-1978 2 or more stories   
6                Masonry Pre-1942 less than 2 stories   
7                  Masonry Pre-1942 2 or more stories   
8                 Frame Post-1978 Less than 2 stories   
9                         Frame Post-1978 Split Level   
10                  Frame Post-1978 2 or more stories   
11                Frame 1942-1978 less than 2 stories   
12                         Frame Pre-1942 Split Level   
13                  Frame 1942-1978 2 or more stories   
14                 Frame Pre-1942 less than 2 stories   
15                   Frame Pre-1942 2 or more stories   

                                                                            \
               Non-normalized gas usage Non-normalized elec. usage   count   
HousingGroupNo                                                               
1                                  1133                      10282   618.0   
2                                  1416                       9661  8950.0   
3                                  1812                      18225   626.0   
4                                  1210                       8779  4310.0   
5                                  1427                      10549  1728.0   
6                                  1504                       9299   625.0   
7                                  2044                      13272  1043.0   
8                                  1054                       9353   637.0   
9                                  1168                      10445  1295.0   
10                                 1422                      13495   658.0   
11                                 1176                       9213  3145.0   
12                                 1297                       9964  2100.0   
13                                 1530                      12543  1237.0   
14                                 1340                       9103   728.0   
15                                 1753                      11967  1372.0   

                                                                            \
                 median gas   median elec Gas Diff-med(%) Elec Diff-med(%)   
HousingGroupNo                                                               
1                701.823110   8710.662051       38.056213        15.282415   
2               1082.151900   9109.631282       23.576843         5.707160   
3               1102.059252  11136.748672       39.179953        38.893011   
4                950.899719   8438.543606       21.413246         3.878077   
5               1459.271962  10335.399225       -2.261525         2.024844   
6                902.389870   7973.776697       40.000674        14.251245   
7               1388.303517  10182.996506       32.079084        23.274589   
8                723.058443   8700.472351       31.398630         6.976667   
9                906.787695   9786.061754       22.364067         6.308648   
10              1121.269569  11175.157751       21.148413        17.190383   
11               951.791907   8573.439108       19.065314         6.941940   
12              1189.091363   9353.199724        8.319864         6.130071   
13              1445.751931  10442.689056        5.506410        16.744885   
14               910.150935   8265.122598       32.078288         9.204410   
15              1347.104100  10218.326847       23.154358        14.612461   

                                           ...  \
                   mean gas     mean elec  ...   
HousingGroupNo                             ...   
1                760.723057   9743.608038  ...   
2           

### Write output to folder

In [450]:
#out_name = "Groups_results" + "_" + baseline_result.split("_")[2]
out.to_csv(os.path.join(result_path, iteration_path, 'baseline_prototype_results.csv'), index=True)
print(f'file saved to:\n  {os.path.join(result_path, iteration_path)}')

file saved to:
  /Users/lliu2/Documents/Chicago retrofits/ResStock results/cookcnty_2007_05/baseline
